In [ ]:
import pandas as pd
import numpy as np
from rep.utils import train_test_split_group
import xgboost as xgb
from sklearn.cross_validation import train_test_split
from scipy.stats import kurtosis,skew, mode
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib

# FUNCTIONS

In [ ]:
def d(x):
    return x[:-9]
def dtw(x):
    return x%7
def get_month(x):
    if 152<x<184:
        return 1
    elif 183<x<212:
        return 2
    elif  211<x<243:
        return 3
    elif  242<x<273:
        return 4
    elif 272<x<304:
        return 5
    elif 303<x<334:
        return 6
    elif 333<x<365:
        return 7
    elif (364<x<396)or(0<=x<31):
        return 8
    elif (395<x<426)or(30<x<61):
        return 9
    elif (425<x<457)or(60<x<92):
        return 10
    elif (91<x<122):
        return 11
    elif 121<x<153:
        return 12
    else:
        return 0
def get_day(x):
    if 152<x<184:
        return x-152
    elif 183<x<212:
        return x-183
    elif  211<x<243:
        return x-211
    elif  242<x<273:
        return x-242
    elif 272<x<304:
        return x-272
    elif 303<x<334:
        return x-303
    elif 333<x<365:
        return x-333
    elif (364<x<396):
        return x-364
    elif(0<=x<31):
        return x+1
    elif (395<x<426):
        return x-395
    elif(30<x<61):
        return x-30
    elif (425<x<457):
        return x-425
    elif(60<x<92):
        return x-60
    elif 121<x<153:
        return x-121
    else:
        return 0
def time(x):
    return x[-8:]
def am_m(x):
    if(x<0):
        return x
    else:
        return 0
def am_p(x):
    if(x>0):
        return x
    else:
        return 0
def am_m_min(x):
    if(x<0):
        return x
    else:
        return -145984526
def am_p_min(x):
    if(x>0):
        return x
    else:
        return 145984526

def count_0(x):
    if x>0:
        return 1
    else:
        return 0
def divide(a, b):
    if b>0:
        return a/b
    else:
        return a

In [ ]:
trans = pd.read_csv("data/transactions.csv")

In [ ]:
trans['amount'] = trans['amount']*(np.e/680)

In [ ]:
trans['day'] = trans['tr_datetime'].apply(d)

In [ ]:
trans['day'] = trans['day'].apply(int)

In [ ]:
trans['week'] = trans['day'].apply(dtw)

In [ ]:
trans['month'] = trans['day'].apply(get_month)

In [ ]:
trans['days'] = trans['day'].apply(get_day)

In [ ]:
trans['datetime'] = trans['tr_datetime'].apply(time)

In [ ]:
trans.drop('tr_datetime',axis=1,inplace=True)

In [ ]:
trans['count_size'] = trans['customer_id'].groupby(trans['customer_id']).transform('size')


In [ ]:
trans['hour'] = trans['datetime'].apply(lambda x: int(x.split(':')[0]))
trans['minute'] = trans['datetime'].apply(lambda x: int(x.split(':')[1]))
#trans['second'] = trans['datetime'].apply(lambda x: int(x.split(':')[2]))

In [ ]:
trans.drop('datetime',axis=1,inplace=True)

In [ ]:
trans['sec'] = 60*60*trans['hour']+60*trans['minute']

In [ ]:
trans['secd'] = 24*60*60*trans['days']+60*60*trans['hour']+60*trans['minute']

In [ ]:
trans['amount_minus'] = -trans['amount'].apply(am_m)
trans['amount_plus'] = trans['amount'].apply(am_p)

In [ ]:
transmed = pd.read_csv("data/mcc_full.csv")

# optimization memory

In [ ]:
trans.info()

In [ ]:
for col in trans.columns:
    if col!='term_id':
        trans[col] = trans[col].astype(np.int32)

In [ ]:
trans.head()

In [ ]:
trans.drop(['term_id'],axis=1,inplace=True)

# Corr

In [ ]:
fig, ax = plt.subplots(figsize=(9, 9))
ax.matshow(trans.corr(), vmin=-1, vmax=1)
plt.yticks(range(trans.shape[1]), trans.columns)
plt.xticks(range(trans.shape[1]), trans.columns, rotation=45, ha="left" )
plt.show()

In [ ]:
import time

# NEW FEATURES

In [ ]:
len(trans.days.unique())

In [ ]:
trans.head()

# MONTH DAY

In [ ]:
trans['sum_am_m_month']=trans.groupby(['customer_id','month'])['amount_minus'].transform('sum') 
trans['sum_am_p_month']=trans.groupby(['customer_id','month'])['amount_plus'].transform('sum')
trans['med_am_m_month']=trans.groupby(['customer_id','month'])['amount_minus'].transform('median') 
trans['med_am_p_month']=trans.groupby(['customer_id','month'])['amount_plus'].transform('median')
trans['mean_am_m_month']=trans.groupby(['customer_id','month'])['amount_minus'].transform('mean') 
trans['mean_am_p_month']=trans.groupby(['customer_id','month'])['amount_plus'].transform('mean')
trans['count_am_m_month'] = trans['amount_minus'].apply(count_0)
trans['count_am_p_month'] = trans['amount_plus'].apply(count_0)
trans['count_am_m_month']=trans.groupby(['customer_id','month'])['count_am_m_month'].transform('sum') 
trans['count_am_p_month']=trans.groupby(['customer_id','month'])['count_am_p_month'].transform('sum') 

In [ ]:
for i in trans.week.unique():
    def dump(x):
        if x==i:
            return 1
        else:
            return 0
    %time trans['month_{}'.format(i)] = trans['month'].apply(dump).astype(np.int32)
    %time trans['month_{}_m_sum'.format(i)] = trans['month_{}'.format(i)]*trans['sum_am_m_month'].astype(np.int32)
    %time trans['month_{}_p_sum'.format(i)] = trans['month_{}'.format(i)]*trans['sum_am_p_month'].astype(np.int32)
    %time trans['month_{}_m_med'.format(i)] = trans['month_{}'.format(i)]*trans['med_am_m_month'].astype(np.int32)
    %time trans['month_{}_p_med'.format(i)] = trans['month_{}'.format(i)]*trans['med_am_p_month'].astype(np.int32)
    %time trans['month_{}_m_mean'.format(i)] = trans['month_{}'.format(i)]*trans['mean_am_m_month'].astype(np.int32)
    %time trans['month_{}_p_mean'.format(i)] = trans['month_{}'.format(i)]*trans['mean_am_p_month'].astype(np.int32)
    %time trans['month_{}_m_count'.format(i)] = trans['month_{}'.format(i)]*trans['count_am_m_month'].astype(np.int32)
    %time trans['month_{}_p_count'.format(i)] = trans['month_{}'.format(i)]*trans['count_am_p_month'].astype(np.int32)
    trans['month_{}_m_sum'.format(i)]=trans.groupby(['customer_id'])['month_{}_m_sum'.format(i)].transform('max') 
    trans['month_{}_p_sum'.format(i)]=trans.groupby(['customer_id'])['month_{}_p_sum'.format(i)].transform('max')
    trans['month_{}_m_med'.format(i)]=trans.groupby(['customer_id'])['month_{}_m_med'.format(i)].transform('max') 
    trans['month_{}_p_med'.format(i)]=trans.groupby(['customer_id'])['month_{}_p_med'.format(i)].transform('max')
    trans['month_{}_m_mean'.format(i)]=trans.groupby(['customer_id'])['month_{}_m_mean'.format(i)].transform('max') 
    trans['month_{}_p_mean'.format(i)]=trans.groupby(['customer_id'])['month_{}_p_mean'.format(i)].transform('max')
    trans['month_{}_m_count'.format(i)]=trans.groupby(['customer_id'])['month_{}_m_count'.format(i)].transform('max') 
    trans['month_{}_p_count'.format(i)]=trans.groupby(['customer_id'])['month_{}_p_count'.format(i)].transform('max')
    trans.drop(['month_{}'.format(i)],axis=1,inplace=True)

In [ ]:
transmed=trans.groupby('customer_id', as_index=False).median()
transmed.to_csv("data/month_day_full.csv",index=False)

# week mean sum median count

In [ ]:
trans['sum_am_m_week']=trans.groupby(['customer_id','week'])['amount_minus'].transform('sum') 
trans['sum_am_p_week']=trans.groupby(['customer_id','week'])['amount_plus'].transform('sum')
trans['med_am_m_week']=trans.groupby(['customer_id','week'])['amount_minus'].transform('median') 
trans['med_am_p_week']=trans.groupby(['customer_id','week'])['amount_plus'].transform('median')
trans['mean_am_m_week']=trans.groupby(['customer_id','week'])['amount_minus'].transform('mean') 
trans['mean_am_p_week']=trans.groupby(['customer_id','week'])['amount_plus'].transform('mean')
trans['count_am_m_week'] = trans['amount_minus'].apply(count_0)
trans['count_am_p_week'] = trans['amount_plus'].apply(count_0)
trans['count_am_m_week']=trans.groupby(['customer_id','week'])['count_am_m_week'].transform('sum') 
trans['count_am_p_week']=trans.groupby(['customer_id','week'])['count_am_p_week'].transform('sum') 

In [ ]:
for i in trans.week.unique():
    def dump(x):
        if x==i:
            return 1
        else:
            return 0
    %time trans['week_{}'.format(i)] = trans['week'].apply(dump).astype(np.int32)
    %time trans['week_{}_m_sum'.format(i)] = trans['week_{}'.format(i)]*trans['sum_am_m_week'].astype(np.int32)
    %time trans['week_{}_p_sum'.format(i)] = trans['week_{}'.format(i)]*trans['sum_am_p_week'].astype(np.int32)
    %time trans['week_{}_m_med'.format(i)] = trans['week_{}'.format(i)]*trans['med_am_m_week'].astype(np.int32)
    %time trans['week_{}_p_med'.format(i)] = trans['week_{}'.format(i)]*trans['med_am_p_week'].astype(np.int32)
    %time trans['week_{}_m_mean'.format(i)] = trans['week_{}'.format(i)]*trans['mean_am_m_week'].astype(np.int32)
    %time trans['week_{}_p_mean'.format(i)] = trans['week_{}'.format(i)]*trans['mean_am_p_week'].astype(np.int32)
    %time trans['week_{}_m_count'.format(i)] = trans['week_{}'.format(i)]*trans['count_am_m_week'].astype(np.int32)
    %time trans['week_{}_p_count'.format(i)] = trans['week_{}'.format(i)]*trans['count_am_p_week'].astype(np.int32)
    trans['week_{}_m_sum'.format(i)]=trans.groupby(['customer_id'])['week_{}_m_sum'.format(i)].transform('max') 
    trans['week_{}_p_sum'.format(i)]=trans.groupby(['customer_id'])['week_{}_p_sum'.format(i)].transform('max')
    trans['week_{}_m_med'.format(i)]=trans.groupby(['customer_id'])['week_{}_m_med'.format(i)].transform('max') 
    trans['week_{}_p_med'.format(i)]=trans.groupby(['customer_id'])['week_{}_p_med'.format(i)].transform('max')
    trans['week_{}_m_mean'.format(i)]=trans.groupby(['customer_id'])['week_{}_m_mean'.format(i)].transform('max') 
    trans['week_{}_p_mean'.format(i)]=trans.groupby(['customer_id'])['week_{}_p_mean'.format(i)].transform('max')
    trans['week_{}_m_count'.format(i)]=trans.groupby(['customer_id'])['week_{}_m_count'.format(i)].transform('max') 
    trans['week_{}_p_count'.format(i)]=trans.groupby(['customer_id'])['week_{}_p_count'.format(i)].transform('max')
    trans.drop(['week_{}'.format(i)],axis=1,inplace=True)

In [ ]:
trans.describe()

# mcc code

In [ ]:
(trans.mcc_code.value_counts().cumsum()/len(trans))[0:2]

In [ ]:
len(trans.mcc_code.unique())

In [ ]:
trans['sum_am_m_mcc_code']=trans.groupby(['customer_id','mcc_code'])['amount_minus'].transform('sum') 
trans['sum_am_p_mcc_code']=trans.groupby(['customer_id','mcc_code'])['amount_plus'].transform('sum')


In [ ]:
s=0
for i in trans.mcc_code.unique()[0:90]:
    print('NEXT')
    print(s)
    def dump(x):
        if x==i:
            return 1
        else:
            return 0
    %time trans['mcc_code_{}'.format(i)] = trans['mcc_code'].apply(dump).astype(np.int32)
    %time trans['mcc_code_{}_m_sum'.format(i)] = trans['mcc_code_{}'.format(i)]*trans['sum_am_m_mcc_code'].astype(np.int32)
    %time trans['mcc_code_{}_p_sum'.format(i)] = trans['mcc_code_{}'.format(i)]*trans['sum_am_p_mcc_code'].astype(np.int32)
    %time trans['mcc_code_{}_m_sum'.format(i)]=trans.groupby(['customer_id'])['mcc_code_{}_m_sum'.format(i)].transform('max').astype(np.int32) 
    %time trans['mcc_code_{}_p_sum'.format(i)]=trans.groupby(['customer_id'])['mcc_code_{}_p_sum'.format(i)].transform('max').astype(np.int32)
    %time trans.drop(['mcc_code_{}'.format(i)],axis=1,inplace=True)
    s=s+1

In [ ]:
max(trans.mcc_code_7699_p_sum)

# 

In [ ]:
trans['sum_am_m_day']=trans.groupby(['customer_id','day'])['amount_minus'].transform('sum') 
trans['sum_am_p_day']=trans.groupby(['customer_id','day'])['amount_plus'].transform('sum')

In [ ]:
trans['sum_am_m_day']=trans.groupby(['customer_id','day'])['amount_minus'].transform('sum') 
trans['sum_am_p_day']=trans.groupby(['customer_id','day'])['amount_plus'].transform('sum')

In [ ]:
trans['sum_am_m_days']=trans.groupby(['customer_id','days'])['amount_minus'].transform('sum') 
trans['sum_am_p_days']=trans.groupby(['customer_id','days'])['amount_plus'].transform('sum')

In [ ]:
trans['sum_am_m_week']=trans.groupby(['customer_id','day'])['amount_minus'].transform(mode)

# featch_stack_one

In [ ]:
trans['sum_am_m']=trans.groupby(['customer_id'])['amount_minus'].transform('sum') 
trans['sum_am_p']=trans.groupby(['customer_id'])['amount_plus'].transform('sum')

In [ ]:
trans['mean_am_m']=trans.groupby(['customer_id'])['amount_minus'].transform('mean') 
trans['mean_am_p']=trans.groupby(['customer_id'])['amount_plus'].transform('mean')

In [ ]:
trans['med_am_m']=trans.groupby(['customer_id'])['amount_minus'].transform('median') 
trans['med_am_p']=trans.groupby(['customer_id'])['amount_plus'].transform('median')

In [ ]:
trans['first_am_m']=trans.groupby(['customer_id'])['amount_minus'].transform('first') 
trans['first_am_p']=trans.groupby(['customer_id'])['amount_plus'].transform('first')

In [ ]:
trans['max_am_m']=trans.groupby(['customer_id'])['amount_minus'].transform('max') 
trans['max_am_p']=trans.groupby(['customer_id'])['amount_plus'].transform('max')

In [ ]:
trans['max_day']=trans.groupby(['customer_id'])['day'].transform('max') 
trans['min_day']=trans.groupby(['customer_id'])['day'].transform('min')
trans['dif_day']=trans['max_day']-trans['min_day']

In [ ]:
trans['amount_minus_min'] = -trans['amount'].apply(am_m_min)
trans['amount_plus_min'] = trans['amount'].apply(am_p_min)
trans['min_am_m']=trans.groupby(['customer_id'])['amount_minus_min'].transform('min') 
trans['min_am_p']=trans.groupby(['customer_id'])['amount_plus_min'].transform('min')

In [ ]:
trans.drop(['amount_minus_min',"amount_plus_min"],axis=1,inplace=True)

In [ ]:
trans['razmah_am_m']=trans["max_am_m"]-trans['min_am_m']
trans['razmah_am_p']=trans["max_am_p"]-trans['min_am_p']

In [ ]:
trans['last_am_m']=trans.groupby(['customer_id'])['amount_minus'].transform('last') 
trans['last_am_p']=trans.groupby(['customer_id'])['amount_plus'].transform('last')

In [ ]:
trans['kurt_am_m']=trans.groupby(['customer_id'])['amount_minus'].transform(kurtosis)
trans['kurt_am_p']=trans.groupby(['customer_id'])['amount_plus'].transform(kurtosis)

In [ ]:
trans['skew_am_m']=trans.groupby(['customer_id'])['amount_minus'].transform(skew)
trans['skew_am_p']=trans.groupby(['customer_id'])['amount_plus'].transform(skew)

In [ ]:
trans['div_m'] = trans['amount_minus'].apply(count_0)
trans['div_p'] = trans['amount_plus'].apply(count_0)

In [ ]:
trans['div_sm']=trans.groupby(['customer_id'])['div_m'].transform('sum') 
trans['div_sp']=trans.groupby(['customer_id'])['div_p'].transform('sum') 

In [ ]:
trans['dif_sum']=trans['div_sm']-trans['div_sp']

In [ ]:
trans['div_smdiv_sp'] = trans.apply(lambda row: divide(row['div_sm'], row['div_sp']), axis=1)

In [ ]:
trans['div_am_sm'] = trans.apply(lambda row: divide(row['sum_am_m'], row['div_sm']), axis=1)

In [ ]:
trans['div_am_sp'] = trans.apply(lambda row: divide(row['sum_am_p'], row['div_sp']), axis=1)

In [ ]:
trans.head()

In [ ]:
trans.info()

In [ ]:
transmed.to_csv('data/feat_1.csv', index=False)

In [ ]:
#trans['size'] = trans.groupby(['customer_id'])['amount'].transform('sum') 

In [ ]:
#trans = pd.merge(trans,gend,on = 'customer_id')

In [ ]:
transmed=trans.groupby('customer_id', as_index=False).median()

In [ ]:
transmean=trans.groupby('customer_id', as_index=False).mean()

In [ ]:
transsum=trans.groupby('customer_id', as_index=False).sum()

In [ ]:
transmax=trans.groupby('customer_id', as_index=False).max()

In [ ]:
transmean=trans.groupby('customer_id', as_index=False).min()

In [ ]:
trans.shape,transmed.shape

In [ ]:
trans.head()

In [ ]:
trans.describe()

In [ ]:
len(trans.mcc_code.unique())

In [ ]:
len(transmed.columns)+len(Xtype.columns)#+len(Xmcc.columns)

In [ ]:
Xtype = trans.groupby('customer_id', as_index=False) \
                    .apply(lambda x: x[['tr_type']].unstack().value_counts()) \
                    .unstack() \
                 .fillna(0)

#, as_index=False

In [ ]:
Xtype = Xtype.rename(columns=lambda x: str(x)+'type')

In [ ]:
Xmcc = trans.groupby('customer_id', as_index=False) \
                    .apply(lambda x: x[['mcc_code']].unstack().value_counts()) \
                    .unstack() \
                 .fillna(0)

In [ ]:
Xmcc = Xmcc.rename(columns=lambda x: str(x)+'mcc')

In [ ]:
Xweek = trans.groupby('customer_id', as_index=False) \
                    .apply(lambda x: x[['week']].unstack().value_counts()) \
                    .unstack() \
                 .fillna(0)

In [ ]:
Xweek = Xweek.rename(columns=lambda x: str(x)+'week')

In [ ]:
Xday = trans.groupby('customer_id', as_index=False) \
                    .apply(lambda x: x[['days']].unstack().value_counts()) \
                    .unstack() \
                 .fillna(0)

In [ ]:
Xday = Xday.rename(columns=lambda x: str(x)+'day')

In [ ]:
Xhour = trans.groupby('customer_id', as_index=False) \
                    .apply(lambda x: x[['hour']].unstack().value_counts()) \
                    .unstack() \
                 .fillna(0)

In [ ]:
Xhour = Xhour.rename(columns=lambda x: str(x)+'hour')

In [ ]:
Xmonth = trans.groupby('customer_id', as_index=False) \
                    .apply(lambda x: x[['month']].unstack().value_counts()) \
                    .unstack() \
                 .fillna(0)

In [ ]:
Xmonth = Xmonth.rename(columns=lambda x: str(x)+'month')

In [ ]:
Xmcc.shape

In [ ]:
Xmcc = pd.concat([transmed,Xmcc],axis=1,ignore_index=False)

In [ ]:
Xmcc = pd.concat([Xmcc,Xtype],axis=1,ignore_index=False)

In [ ]:
Xmcc = pd.concat([Xmcc,Xweek],axis=1,ignore_index=False)

In [ ]:
Xmcc = pd.concat([Xmcc,Xday],axis=1,ignore_index=False)

In [ ]:
Xmcc = pd.concat([Xmcc,Xhour],axis=1,ignore_index=False)

In [ ]:
Xmcc = pd.concat([Xmcc,Xmonth],axis=1,ignore_index=False)

# TRAIN TEST

In [ ]:
gend = pd.read_csv('data/customers_gender_train.csv')

In [ ]:
customers_gender = gend.set_index('customer_id')

In [ ]:
Xmcc.info()

In [ ]:
Xmcc = Xmcc.groupby('customer_id').mean()

In [ ]:
Y_train = customers_gender.loc[Xmcc.index].gender
Y_train = Y_train.reset_index()
del Y_train['customer_id']
Y_train = Y_train.dropna(0)

In [ ]:
X_train = Xmcc.reset_index()
X_train = X_train.loc[Y_train.index]
X_train['gender'] = Y_train
X_train.head()

In [ ]:
X_test = Xmcc.drop(customers_gender.index)
X_test.head()

In [ ]:
#X_test = Xmcc.drop(customers_gender.index).reset_index()
#X_test.head()

In [ ]:
feat = list(set(X_test.columns)-{'customer_id'})
len(feat)

In [ ]:
#X_train, Xtest, Y_train, Ytest = train_test_split(X_train, X_train.gender, test_size=0.15, random_state=7)

In [ ]:
Xtr, Xval, ytr, yval = train_test_split(X_train, X_train.gender, test_size=0.2, random_state=7)

# MODELING

# RESULT

In [ ]:
#noweek+week_sum_notype 0.873=800
#base 0.869 = 800
#mcc tr mccfull 0.877338

In [ ]:
dtr = xgb.DMatrix(Xtr[feat], label=ytr)
dval = xgb.DMatrix(Xval[feat], label=yval)
watchlist = [(dtr, 'train'), (dval, 'eval')]
history = dict()

In [ ]:
params = {
    'max_depth': 4,
    'eta': 0.025,
    'nthread': 4,
    'subsample': 0.8,
    'eval_metric': ['auc'],
    'objective': 'binary:logistic',
    'colsample_bytree': 0.6,
    'min_child_weight': 0,
    #'scale_pos_weight':(1-train.Label.mean())/train.Label.mean(),
    'seed':7
}

In [ ]:
#noweek+week_sum_notype
xgb.cv(params, dtr, 1000, nfold=5,
       metrics={'auc'}, seed = 7,verbose_eval=50,
       callbacks=[xgb.callback.print_evaluation(show_stdv=True)])

In [ ]:
0849 0.025
086 0.05
0861 0.1
0877
08639 0.075

In [ ]:
model = xgb.train(params, dtr, num_boost_round=1000, evals=watchlist,evals_result=history, verbose_eval=50)

In [ ]:
imps = pd.Series(model.get_fscore()).sort_values()
imps.plot(kind='bar', figsize=(15,6),logy=True)

In [ ]:
importn=model.get_fscore()

In [ ]:
importn

In [ ]:
features = []
for i in Xtr.columns:
    try:
        if(importn.get(i)>5):
            features.append(i)
    except:
        s=0

In [ ]:
imps.plot(kind='bar', figsize=(15,6),logy=True)

In [ ]:
dtest = xgb.DMatrix(X_test[feat])

In [ ]:
result = pd.DataFrame(X_test.index, columns=['customer_id'])
result['gender'] = model.predict(dtest)

In [ ]:
result.to_csv('a_xgb_feat1.csv', index=False)

In [ ]:
result

# dataprep

In [ ]:
transmed.to_csv('data/mcc_full.csv', index=False)

In [ ]:
transmed.head()

# MetaFeatches

In [ ]:
from sklearn.cross_validation import KFold
X_train['Proba']=0
X_test['Proba']=0
kf = KFold(X_train.shape[0], n_folds=5,shuffle=True)
for train_index, val_index in kf:
    #print("TRAIN:", train_index, "TEST:", val_index)
    print("!!!!!!!!!!!!!!!!!!!!!!Next Fold!!!!!!!!!!!!")
    training_data0, validation_data0 = X_train.loc[train_index], X_train.loc[val_index]
    dtr = xgb.DMatrix(training_data0[features], label=training_data0.gender)
    dval = xgb.DMatrix(validation_data0[features], label=validation_data0.gender)
    watchlist = [(dtr, 'train'), (dval, 'eval')]
    history = dict()
    model0 = xgb.train(params, dtr, num_boost_round=1000, evals=watchlist,evals_result=history, verbose_eval=50)

    X_train['Proba'].loc[val_index] = model0.predict(dval)
    X_test['Proba'] += model0.predict(dtest)
    
X_test['Proba'] = X_test['Proba']/kf.n_folds

In [ ]:
features

In [ ]:
features.append('Proba')

In [ ]:
features.remove('proba')

In [ ]:
#data = pd.merge(trans,gend,on = 'customer_id')

In [ ]:
#data.head()

In [ ]:
#data = pd.get_dummies(data, columns=['mcc_code'])

In [ ]:
Xtr, Xval = train_test_split_group(data.customer_id, data, random_state=7, train_size=0.8)

In [ ]:
gbm = xgb.XGBClassifier()
dtrain = xgb.DMatrix(Xtr[features], label=Xtr.gender)

In [ ]:
len(trans[trans['count_size']>2000]['customer_id'].unique())

In [ ]:
z = trans[trans['count_size']<3000]
z.describe()